In [1]:
# Standard Imports for all notebooks
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
# Other Quality of Life Imports
from time import time
from datetime import datetime
datetime.fromtimestamp(time()).strftime('%m/%d/%Y, %H:%M:%S')

import warnings
warnings.filterwarnings('ignore')

In [3]:
# Selenium
# Scraping Libraries
from bs4 import BeautifulSoup
import requests
from time import sleep
import re


In [4]:
get_dex = requests.get("https://bulbapedia.bulbagarden.net/wiki/List_of_Pok%C3%A9mon_by_Blueberry_Pok%C3%A9dex_number")

In [5]:
html_dex = BeautifulSoup(get_dex.text, "lxml")

In [9]:
df = pd.read_html(get_dex.text)[0]

In [12]:
df = df.drop(["MS","Type","Type.1","Image"], axis=1)

In [19]:
df["Ndex"] = df["Ndex"].map(lambda x: int(x.replace("#","")))

In [23]:
df.sort_values("Ndex")

,BBdex,Ndex,Pokémon
179,#164,1,Bulbasaur
180,#165,2,Ivysaur
181,#166,3,Venusaur
182,#167,4,Charmander
183,#168,5,Charmeleon
...,...,...,...
252,#237,1021,Raging Bolt
254,#239,1022,Iron Boulder
253,#238,1023,Iron Crown
255,#240,1024,Terapagos


In [15]:
list_dex = []
for num in range(1,5):
    for str_poke in pd.read_html(get_dex.text)[num]["Pokémon"]:
        list_dex.append(str_poke)

In [16]:
# Original Version
list_dex = [x.replace(" ", "-"
            ).replace(".",""
            ).replace("'",""
            ).replace("Normal", ""
            #).replace("shelloswest-sea","shellos"
            #).replace("plant-cloak",""
            ).lower() for x in list_dex if "Form" not in x]
list_dex.append(["burmy", "shellos", "gastrodon"])

In [17]:
dict_pokemon = {}
for pokemon in list_dex:
    try:
        json_species = requests.get("https://pokeapi.co/api/v2/pokemon-species/" + pokemon).json()
        json_encounters = requests.get("https://pokeapi.co/api/v2/pokemon/" + pokemon + "/encounters").json()
        dict_pokemon[pokemon] = {"Gender Rate"  : json_species["gender_rate"],
                                     "Encounters"   : json_encounters,
                                     "Evolves From" : json_species["evolves_from_species"]}
    except:
        print(pokemon, " was skipped.")

nidoran♀  was skipped.
nidoran♂  was skipped.
unownone-form  was skipped.
kyogrekyogre  was skipped.
groudongroudon  was skipped.
burmyplant-cloak  was skipped.
burmysandy-cloak  was skipped.
burmytrash-cloak  was skipped.
wormadamplant-cloak  was skipped.
wormadamsandy-cloak  was skipped.
wormadamtrash-cloak  was skipped.
shelloswest-sea  was skipped.
shelloseast-sea  was skipped.
gastrodonwest-sea  was skipped.
gastrodoneast-sea  was skipped.
rotomrotom  was skipped.
rotomheat-rotom  was skipped.
rotomwash-rotom  was skipped.
rotomfrost-rotom  was skipped.
rotomfan-rotom  was skipped.
rotommow-rotom  was skipped.
arceusarceus  was skipped.
['burmy', 'shellos', 'gastrodon']  was skipped.


In [18]:
#dict_row = {}
list_row_dicts = []
for key_one_pokemon in list(dict_pokemon.keys()):
    for dict_two_encounter in dict_pokemon[key_one_pokemon]["Encounters"]:
        #print(dict_two_encounter)
        #for dict_ugh_version in dict_two_encounter:
        #    print("-----")
        #    print(dict_ugh_version)
        for dict_tre_version in dict_two_encounter["version_details"]:#["encounter_details"]:
            #print("---")
            #print(dict_tre_version)
            for dict_for_ent_details in dict_tre_version["encounter_details"]:
                #print("---")
                #print(dict_for_ent_details)
                #print(dict_for_ent_details.keys())
                dict_row = {
                    "Pokemon"      : key_one_pokemon,
                    "Gender"       : dict_pokemon[key_one_pokemon]["Gender Rate"],
                    "Location"     : dict_two_encounter["location_area"]["name"],
                    #"Max_Chance"   : dict_tre_version["max_chance"],
                    "Version"      : dict_tre_version["version"]["name"],
                    "Chance"       : dict_for_ent_details["chance"],
                    "Condition"    : dict_for_ent_details["condition_values"],#["name"],
                    "Max Level"    : dict_for_ent_details["max_level"],
                    "Min Level"    : dict_for_ent_details["min_level"],
                    "Method"       : dict_for_ent_details["method"]["name"],
                    "Evolves From" : dict_pokemon[key_one_pokemon]["Evolves From"]
                }
            list_row_dicts.append(dict_row)
df = pd.DataFrame(list_row_dicts)
#df.head(20)

In [20]:
df_cute = df[(df["Gender"] > 0) &
  (df["Gender"] != 8)]

# Bring your own data prep

In [21]:
df_cute = df_cute[(df_cute["Version"] == "soulsilver")|
    (df_cute["Version"] == "heartgold" )|
    (df_cute["Version"] == "pearl"    )| 
    (df_cute["Version"] == "diamond"   )|
    (df_cute["Version"] == "platinum")  ]

In [22]:
df_cute["Method"].value_counts()

walk             3819
surf              768
super-rod         675
good-rod          372
gift               40
rock-smash         36
only-one           17
gift-egg           12
pokeflute           4
old-rod             2
squirt-bottle       2
Name: Method, dtype: int64

In [23]:
df_cute[df_cute["Method"] == "old-rod"]

,Pokemon,Gender,Location,Version,Chance,Condition,Max Level,Min Level,Method,Evolves From
8764,magikarp,4,johto-safari-zone-zone-wetland,heartgold,10,[],15,15,old-rod,None
8765,magikarp,4,johto-safari-zone-zone-wetland,soulsilver,10,[],15,15,old-rod,None


In [24]:
df_cute = df_cute[(df_cute["Method"] != "gift") &
(df_cute["Method"] != "gift-egg")]

In [25]:
def fun_temp(value_in):
    if value_in == None:
        value_out = None
    else:
        value_out = value_in["name"]
    return value_out

df_cute["Evolves From"] = df_cute["Evolves From"].map(lambda x: fun_temp(x)) 

In [26]:
def fun_temp(value_in):
    if len(value_in) == 0:
        value_out = None
    elif len(value_in) == 1:
        value_out = value_in[0]["name"]
    else:
        value_out = value_in[0]["name"] + ", " + value_in[1]["name"]
    return value_out

df_cute["Condition"] = df_cute["Condition"].map(lambda x: fun_temp(x))

def fun_temp(value_in):
    value_out = "ERROR"
    if type(value_in) == type(None):
        value_out = None
    elif value_in.count("slot2") == 1:
        value_out = value_in
    else:
        value_out = None
    return value_out


df_cute["Condition"] = df_cute["Condition"].map(lambda x: fun_temp(x))

In [27]:
df_cute["Gender"] = df_cute["Gender"].map(lambda x: x/8)

In [28]:
df_cute.head()

,Pokemon,Gender,Location,Version,Chance,Condition,Max Level,Min Level,Method,Evolves From
27,caterpie,0.5,eterna-forest-area,platinum,4,slot2-firered,13,13,walk,None
28,caterpie,0.5,sinnoh-route-204-south-towards-jubilife-city,diamond,4,slot2-firered,6,6,walk,None
29,caterpie,0.5,sinnoh-route-204-south-towards-jubilife-city,pearl,4,slot2-firered,6,6,walk,None
30,caterpie,0.5,sinnoh-route-204-south-towards-jubilife-city,platinum,4,slot2-firered,5,5,walk,None
31,caterpie,0.5,sinnoh-route-204-north-towards-floaroma-town,platinum,4,slot2-firered,10,10,walk,None


In [27]:
df_cute_share = df_cute.drop(["Max Level", "Min Level"], axis = 1)#, "Location", "Chance", "Method")

df_cute_share = pd.concat([df_cute_share.drop(["Version", "Condition"], axis = 1), 
           pd.get_dummies(df_cute_share["Version"]), 
           pd.get_dummies(df_cute_share["Condition"])], axis = 1
             ).drop("slot2-none", axis = 1)

df_cute_share = df_cute_share.groupby('Pokemon').max()

df_cute_share = df_cute_share.reset_index()

df_cute_share["Location"] = df_cute_share["Location"].map(lambda x: x.replace("-", " "))

#df_cute_share.drop(["Max Level", "Location", "Min Level", "Chance", "Method"], axis = 1#)
df_cute_share.to_csv("./data/Cute_Availability.csv", index = False)

In [74]:
list_arrays = []
for mon in df_cute_catch["Pokemon"].unique():
    list_arrays.append(df_cute_catch[df_cute_catch["Pokemon"] == mon].sum())

df_cute_catch = pd.DataFrame(list_arrays)

def fun_singularize(word_in):
    word_out = ""
    if type(word_in) != str:
        word_out = word_in
    else:
        for i in range(1, len(word_in) + 1):
            word = word_in[:i]
            count = word_in.count(word)
            if count * len(word) == len(word_in):
                word_out = word
                break
    return word_out

df_cute_catch["Pokemon"] = df_cute_catch["Pokemon"].map(lambda x: fun_singularize(x))
df_cute_catch["Evolves From"] = df_cute_catch["Evolves From"].map(lambda x: fun_singularize(x))

In [208]:
import pandas as pd

# Create the initial DataFrame
data = {
    'Pokemon': ['abomasnow', 'abomasnow', 'heatran', 'heatran', 'heatran'],
    'Col2': [0.5, 0.5, 0.5, 0.5, 0.5],
    'Col3': [0, 0, 1, 0, 0],
    'Col4': [0, 0, 0, 0, 0],
    'Col5': [1, 0, 0, 1, 0],
    'Col6': [0, 1, 0, 0, 1]
}

df_cute = pd.DataFrame(data)

# Group by 'Pokemon' and sum the values for each group
result_df_cute = df_cute.groupby('Pokemon').sum()

# Reset the index to make 'Pokemon' a regular column
result_df_cute = result_df_cute.reset_index()

# Rename the columns
result_df_cute.columns = ['Pokemon', 'Col2', 'Col3', 'Col4', 'Col5', 'Col6']

print(result_df_cute)


     Pokemon  Col2  Col3  Col4  Col5  Col6
0  abomasnow   1.0     0     0     1     1
1    heatran   1.5     1     0     1     1


## Personal Prep

In [19]:
df_cute.head(10)

,Pokemon,Gender,Location,Version,Chance,Condition,Max Level,Min Level,Method,Evolves From
27,caterpie,0.5,eterna-forest-area,platinum,4,slot2-firered,13,13,walk,None
28,caterpie,0.5,sinnoh-route-204-south-towards-jubilife-city,diamond,4,slot2-firered,6,6,walk,None
29,caterpie,0.5,sinnoh-route-204-south-towards-jubilife-city,pearl,4,slot2-firered,6,6,walk,None
30,caterpie,0.5,sinnoh-route-204-south-towards-jubilife-city,platinum,4,slot2-firered,5,5,walk,None
31,caterpie,0.5,sinnoh-route-204-north-towards-floaroma-town,platinum,4,slot2-firered,10,10,walk,None
34,caterpie,0.5,johto-route-30-area,heartgold,1,None,4,4,walk,None
37,caterpie,0.5,johto-route-31-area,heartgold,1,None,5,5,walk,None
40,caterpie,0.5,ilex-forest-area,heartgold,10,None,6,6,walk,None
45,caterpie,0.5,national-park-area,heartgold,10,None,12,12,walk,None
46,caterpie,0.5,unknown-all-bugs-area,heartgold,1,None,14,14,walk,None


In [21]:
len(df_cute["Pokemon"].unique())

259

In [24]:
df_cute["Evolves From"].unique()[1:]

array(['caterpie', 'metapod', 'weedle', 'kakuna', 'pidgey', 'rattata',
       'spearow', 'ekans', 'pichu', 'sandshrew', 'cleffa', 'igglybuff',
       'zubat', 'oddish', 'paras', 'venonat', 'diglett', 'meowth',
       'psyduck', 'mankey', 'poliwag', 'abra', 'machop', 'bellsprout',
       'tentacool', 'geodude', 'graveler', 'ponyta', 'slowpoke', 'doduo',
       'seel', 'grimer', 'gastly', 'haunter', 'drowzee', 'krabby',
       'cubone', 'koffing', 'rhyhorn', 'horsea', 'goldeen', 'elekid',
       'magby', 'magikarp', 'munchlax', 'dratini', 'sentret', 'hoothoot',
       'ledyba', 'spinarak', 'chinchou', 'mareep', 'azurill', 'marill',
       'bonsly', 'hoppip', 'wooper', 'wynaut', 'onix', 'teddiursa',
       'slugma', 'swinub', 'remoraid', 'mantyke', 'houndour', 'phanpy',
       'larvitar', 'poochyena', 'zigzagoon', 'wurmple', 'silcoon',
       'cascoon', 'lotad', 'seedot', 'taillow', 'wingull', 'ralts',
       'surskit', 'whismur', 'meditite', 'budew', 'carvanha', 'wailmer',
       'numel'

In [ ]:
list_inner = []


In [ ]:
for row in df_cute.
print()

In [29]:
df_cute = df_cute[ 
    (df_cute["Method"] == "walk") |
    (df_cute["Method"] == "surf") |
    (df_cute["Method"] == "super-rod") |
    (df_cute["Method"] == "good-rod")]


In [30]:
def fun_temp(value_in):
    if len(value_in) == 0:
        value_out = None
    elif len(value_in) == 1:
        value_out = value_in[0]["name"]
    else:
        value_out = value_in[0]["name"] + ", " + value_in[1]["name"]
    return value_out

df_cute["Condition"] = df_cute["Condition"].map(lambda x: fun_temp(x))
                    
                    
                    
                    #.value_counts()

In [31]:
df_cute.to_csv("./data/Cute_Pokemon_Data.csv", index = False)

In [ ]:
# Personal Availabilities

df_cute = df_cute[(df_cute["Version"] == "pearl") |
    (df_cute["Version"] == "soulsilver")]

df_cute = df_cute[(df_cute["Condition"] != "slot2-leafgreen") &
    (df_cute["Condition"] != "slot2-emerald") &
    (df_cute["Condition"] != "slot2-sapphire")]

In [26]:
print(len(df_cute["Pokemon"].unique()))
df_cute["Pokemon"].unique()

228


array(['caterpie', 'metapod', 'weedle', 'kakuna', 'beedrill', 'pidgey',
       'pidgeotto', 'rattata', 'raticate', 'spearow', 'fearow', 'ekans',
       'arbok', 'pikachu', 'sandshrew', 'sandslash', 'clefairy', 'vulpix',
       'jigglypuff', 'zubat', 'golbat', 'oddish', 'gloom', 'paras',
       'parasect', 'venonat', 'venomoth', 'diglett', 'dugtrio', 'meowth',
       'persian', 'psyduck', 'golduck', 'mankey', 'primeape', 'growlithe',
       'poliwag', 'poliwhirl', 'abra', 'kadabra', 'machop', 'machoke',
       'bellsprout', 'weepinbell', 'tentacool', 'tentacruel', 'geodude',
       'graveler', 'golem', 'ponyta', 'rapidash', 'slowpoke', 'slowbro',
       'farfetchd', 'doduo', 'dodrio', 'seel', 'dewgong', 'grimer', 'muk',
       'shellder', 'gastly', 'haunter', 'onix', 'drowzee', 'hypno',
       'krabby', 'kingler', 'cubone', 'marowak', 'lickitung', 'koffing',
       'weezing', 'rhyhorn', 'rhydon', 'tangela', 'horsea', 'seadra',
       'goldeen', 'seaking', 'mr-mime', 'electabuzz', 'magma

In [23]:
df_cute[df_cute["Pokemon"] == "nidorino"]

,Pokemon,Gender,Location,Version,Chance,Condition,Max Level,Min Level,Method,Evolves From
1328,nidorino,0,valor-lakefront-area,pearl,10,radar-on,22,22,walk,nidoran-m
1331,nidorino,0,sinnoh-route-221-area,pearl,10,radar-on,29,29,walk,nidoran-m
1338,nidorino,0,kanto-route-13-area,soulsilver,10,radio-off,23,23,walk,nidoran-m
1343,nidorino,0,kanto-route-14-area,soulsilver,10,radio-off,23,23,walk,nidoran-m
1348,nidorino,0,kanto-route-15-area,soulsilver,10,radio-off,23,23,walk,nidoran-m
1367,nidorino,0,johto-safari-zone-zone-savannah,soulsilver,10,time-night,17,17,walk,nidoran-m


In [24]:
df_cute[df_cute["Gender"] == 0]

,Pokemon,Gender,Location,Version,Chance,Condition,Max Level,Min Level,Method,Evolves From
1328,nidorino,0,valor-lakefront-area,pearl,10,radar-on,22,22,walk,nidoran-m
1331,nidorino,0,sinnoh-route-221-area,pearl,10,radar-on,29,29,walk,nidoran-m
1338,nidorino,0,kanto-route-13-area,soulsilver,10,radio-off,23,23,walk,nidoran-m
1343,nidorino,0,kanto-route-14-area,soulsilver,10,radio-off,23,23,walk,nidoran-m
1348,nidorino,0,kanto-route-15-area,soulsilver,10,radio-off,23,23,walk,nidoran-m
1367,nidorino,0,johto-safari-zone-zone-savannah,soulsilver,10,time-night,17,17,walk,nidoran-m
7917,tauros,0,sinnoh-route-209-area,pearl,1,radar-on,16,16,walk,None
7919,tauros,0,sinnoh-route-210-south-towards-solaceon-town,pearl,1,radar-on,20,20,walk,None
7925,tauros,0,johto-route-38-area,soulsilver,4,None,13,13,walk,None
7930,tauros,0,johto-route-39-area,soulsilver,1,None,15,15,walk,None


In [19]:
df_cute["Condition"].value_counts()

radio-hoenn                211
time-night                 155
radio-sinnoh               145
time-day                   138
radio-off                  115
radar-off                  108
radar-on                    49
swarm-yes                   39
time-night, radio-off       31
time-day, radio-off         18
time-morning                15
swarm-no                    14
slot2-ruby                  14
slot2-firered               13
time-morning, radio-off      5
swarm-no, time-day           2
swarm-no, time-night         1
Name: Condition, dtype: int64

In [35]:
df_cute["Method"].value_counts()

walk             3819
surf              768
super-rod         675
good-rod          372
rock-smash         36
only-one           17
pokeflute           4
old-rod             2
squirt-bottle       2
Name: Method, dtype: int64

In [20]:
def fun_temp(value_in):
    if value_in == None:
        value_out = None
    else:
        value_out = value_in["name"]
    return value_out

df_cute["Evolves From"] = df_cute["Evolves From"].map(lambda x: fun_temp(x)) 

In [4]:
df_cute = pd.read_csv("./data/Cute_Pokemon_Data.csv")

In [33]:
df_cute[df_cute["Pokemon"] == "munchlax"]

,Pokemon,Gender,Location,Version,Chance,Condition,Max Level,Min Level,Method,Evolves From


In [10]:
df_cute["Pokemon"].unique()

array(['caterpie', 'metapod', 'butterfree', 'weedle', 'kakuna',
       'beedrill', 'pidgey', 'pidgeotto', 'rattata', 'raticate',
       'spearow', 'fearow', 'ekans', 'arbok', 'pikachu', 'raichu',
       'sandshrew', 'sandslash', 'clefairy', 'vulpix', 'jigglypuff',
       'wigglytuff', 'zubat', 'golbat', 'oddish', 'gloom', 'paras',
       'parasect', 'venonat', 'venomoth', 'diglett', 'dugtrio', 'meowth',
       'persian', 'psyduck', 'golduck', 'mankey', 'primeape', 'growlithe',
       'poliwag', 'poliwhirl', 'poliwrath', 'abra', 'kadabra', 'machop',
       'machoke', 'bellsprout', 'weepinbell', 'tentacool', 'tentacruel',
       'geodude', 'graveler', 'golem', 'ponyta', 'rapidash', 'slowpoke',
       'slowbro', 'farfetchd', 'doduo', 'dodrio', 'seel', 'dewgong',
       'grimer', 'muk', 'shellder', 'cloyster', 'gastly', 'haunter',
       'gengar', 'onix', 'drowzee', 'hypno', 'krabby', 'kingler',
       'exeggcute', 'cubone', 'marowak', 'lickitung', 'koffing',
       'weezing', 'rhyhorn', '

In [11]:
df_cute["Method"].unique()

array(['walk', 'surf', 'good-rod', 'super-rod'], dtype=object)

In [11]:
pd.read_csv("./data/Cute_Availability.csv")

,Pokemon,Gender,Location,Chance,Method,Evolves From,diamond,heartgold,pearl,platinum,soulsilver,slot2-emerald,slot2-firered,slot2-leafgreen,slot2-ruby,slot2-sapphire
0,abomasnow,0.50,mt coronet exterior snowfall,20,walk,snover,1,0,1,1,0,0,0,0,0,0
1,abra,0.25,sinnoh route 215 area,10,walk,NaN,1,1,1,1,1,0,0,0,0,0
2,absol,0.50,whirl islands b3f,20,walk,NaN,1,1,1,1,1,0,0,0,0,0
3,arbok,0.50,kanto route 4 area,10,walk,ekans,1,1,1,1,1,0,1,0,0,0
4,ariados,0.50,sinnoh route 229 area,10,walk,spinarak,1,1,1,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
254,wurmple,0.50,sinnoh route 205 east towards eterna city,10,walk,NaN,1,0,1,1,0,0,0,0,0,0
255,yanma,0.50,johto route 35 area,1,walk,NaN,0,1,0,1,1,0,0,0,0,0
256,zangoose,0.50,sinnoh route 210 west towards celestic town,4,walk,NaN,1,0,1,1,0,0,0,0,1,0
257,zigzagoon,0.50,unknown all rattata area,20,walk,NaN,1,1,1,1,1,0,0,0,0,0


## Pokemon to be catched before returning Ruby

In [106]:
#dict_row = {}
list_row_dicts = []
for key_one_pokemon in list(dict_pokemon.keys()):
    for dict_two_encounter in dict_pokemon[key_one_pokemon]["Encounters"]:
        #print(dict_two_encounter)
        #for dict_ugh_version in dict_two_encounter:
        #    print("-----")
        #    print(dict_ugh_version)
        for dict_tre_version in dict_two_encounter["version_details"]:#["encounter_details"]:
            #print("---")
            #print(dict_tre_version)
            for dict_for_ent_details in dict_tre_version["encounter_details"]:
                #print("---")
                #print(dict_for_ent_details)
                #print(dict_for_ent_details.keys())
                dict_row = {
                    "Pokemon"      : key_one_pokemon,
                    "Gender"       : dict_pokemon[key_one_pokemon]["Gender Rate"],
                    "Location"     : dict_two_encounter["location_area"]["name"],
                    #"Max_Chance"   : dict_tre_version["max_chance"],
                    "Version"      : dict_tre_version["version"]["name"],
                    "Chance"       : dict_for_ent_details["chance"],
                    "Condition"    : dict_for_ent_details["condition_values"],#["name"],
                    "Max Level"    : dict_for_ent_details["max_level"],
                    "Min Level"    : dict_for_ent_details["min_level"],
                    "Method"       : dict_for_ent_details["method"]["name"],
                    "Evolves From" : dict_pokemon[key_one_pokemon]["Evolves From"]
                }
            list_row_dicts.append(dict_row)
df = pd.DataFrame(list_row_dicts)
#df.head(20)

In [34]:
def fun_temp(value_in):
    if value_in == None:
        value_out = None
    else:
        value_out = value_in["name"]
    return value_out

df["Evolves From"] = df["Evolves From"].map(lambda x: fun_temp(x)) 

TypeError: string indices must be integers

In [35]:
df_catch = df.drop(["Gender","Location", "Chance", "Condition", "Max Level", "Min Level", "Method"], axis = 1)

In [36]:
df_catch = pd.concat([df_catch, pd.get_dummies(df_catch["Version"])], axis = 1)

In [42]:
df_catch = df_catch.drop("Version", axis = 1).drop_duplicates()

In [74]:
list_arrays = []
for mon in df_catch["Pokemon"].unique():
    list_arrays.append(df_catch[df_catch["Pokemon"] == mon].sum())

In [76]:
df_catch = pd.DataFrame(list_arrays)

In [96]:
def fun_singularize(word_in):
    word_out = ""
    if type(word_in) != str:
        word_out = word_in
    else:
        for i in range(1, len(word_in) + 1):
            word = word_in[:i]
            count = word_in.count(word)
            if count * len(word) == len(word_in):
                word_out = word
                break
    return word_out

test = "blahblahblah"
test_2 = "bulbasaurbulbasaurbulbasaurbulbasaurbulbasaurbulbasaurbulbasaurbulbasaurbulbasaur"

result1 = fun_singularize(test)
result2 = fun_singularize(test_2)

#print(result1)  # Output: "blah"
#print(result2)  # Output: "bulbasaur"
df_catch["Pokemon"] = df_catch["Pokemon"].map(lambda x: fun_singularize(x))
df_catch["Evolves From"] = df_catch["Evolves From"].map(lambda x: fun_singularize(x))

In [109]:
fun_singularize("bulbasaurbulbasaurbulbasaurbulbasaurbulbasaurbulbasaurbulbasaurbulbasaurbulbasaur")

'bulbasaur'

In [101]:
df_catch.columns

Index(['Pokemon', 'Evolves From', 'alpha-sapphire', 'black', 'black-2', 'blue',
       'crystal', 'diamond', 'emerald', 'firered', 'gold', 'heartgold',
       'leafgreen', 'omega-ruby', 'pearl', 'platinum', 'red', 'ruby',
       'sapphire', 'silver', 'soulsilver', 'sun', 'white', 'white-2', 'x', 'y',
       'yellow'],
      dtype='object')

In [105]:
df_catch[(df_catch["ruby"] == 1) &
         ((df_catch["pearl"] == 0) & 
          (df_catch["soulsilver"] == 0))][['Pokemon', 'Evolves From', 'firered', 'pearl', 'ruby', 'soulsilver']]["Pokemon"]#.unique()

147         xatu
175    heracross
217      silcoon
226      taillow
234    shroomish
242     hariyama
249         aron
250       lairon
254    manectric
276      altaria
286      claydol
287       lileep
288      anorith
289       feebas
291     castform
292      kecleon
293      shuppet
297      tropius
300       wynaut
314     regirock
315       regice
316    registeel
317       latias
Name: Pokemon, dtype: object

In [ ]:
226      taillow
234    shroomish
242     hariyama
249         aron
250       lairon
254    manectric
276      altaria
286      claydol
287       lileep
288      anorith
291     castform
292      kecleon
293      shuppet
297      tropius
300       wynaut


## Recycling

### Solving for pokemon who's post evolution can be caught, but not their pre-evolution.

In [22]:
for mon in df[type(df["Evolves From"]) == None]["Pokemon"].unique():
    if mon not in df["Evolves From"].unique():
        print(mon)

KeyError: False

In [ ]:
df["Evolves From"].dropna().unique()

In [ ]:
df.isna().sum()

In [ ]:
df["Condition"] = df["Condition"].fillna("No Condition")

In [ ]:
list_has_evo_options = list(df.dropna()["Pokemon"].unique())

In [ ]:
list_prevolutions = list(df["Evolves From"].dropna().unique())

In [ ]:
x for x in df["Pokemlist_prevolutions

In [ ]:
https://www.kaggle.com/aravrs/titanic-models-benchmark

In [ ]:
sns.heatmap(train_df.corr(), annot=True, cmap='RdBu_r',
            center=0, vmin=-1, vmax=1, linewidth=2, annot_kws={"fontsize":12},
            square=False, cbar=True).set_title('Correlation matrix', fontsize=24);
plt.yticks(rotation=0);

# Most recent Version
list_dex = [x.replace("-", ""
            ).replace(" ", "-"
            ).replace(".",""
            ).replace("'",""
            ).replace("Normal", ""
            ).replace("eastsea",""
            ).replace("plantcloak",""
            ).lower() for x in list_dex if "Form" not in x]
# Remove UnknownOne form
# Swap "CastformNormal" for "Castform"
# Swap "Burmy" MAYBE.
# Swap "Shellos Sea"'s for Shellos
# Swap "Gastrodon Sea"'s for Gastrodon
# Remove Rotom

In [44]:
list_dex = [str(i) for i in range(1, 494)]
dict_big_pokemon = {}
for pokemon in list_dex:
    #pokemon = pokemon#.replace(" ", "-").replace(".","").replace("'","").replace("Normal", "").lower()
    try:
        dict_game = requests.get("https://pokeapi.co/api/v2/pokemon/" + pokemon).json()
        dict_species = requests.get("https://pokeapi.co/api/v2/pokemon-species/" + pokemon).json()
        dict_big_pokemon[pokemon] = {"Game" : dict_game,
                                     "Species" : dict_species}
    except:
        print(pokemon, "was skipped.")

22  was skipped.
